<a href="https://colab.research.google.com/github/lee1201zxc/study/blob/main/AI/ML/05_3_Ensemble_Tree_GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 앙상블

정형 데이터와 비정형 데이터(표현, 학습 어렵 -> 딥러닝으로)가 존재

여러 모델을 결합해서 더 강한 모델 만듬
###배깅
같은 모델 여러 개를 독립적으로 학습
###부스팅
이전 모델의 실수를 다음 모델이 보완

| 모델                   | 앙상블 방식 |
| -------------------- | ------ |
| RandomForest         | 배깅     |
| ExtraTrees           | 배깅(강화) |
| GradientBoosting     | 부스팅    |
| HistGradientBoosting | 부스팅    |
| XGBoost / LGBM       | 부스팅    |


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 랜덤 포레스트

랜덤으로 샘플, 특성을 선택해서 과적합 방지
기본적으로 100개의 결정 트리 훈련

결정 트리를 랜덤하게 만들어 숲을 만들고 각각 예측해 최종 예측을 만듬

샘플도, 특성도 랜덤 선택(전체 특성에 루트한 값)

오버피팅 방지에 좋음
<br><br>
부트스트랩 샘플
->데이터에서 중복을 허용하여 데이터를 샘플링

In [6]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']
train_input, test_input, train_target, test_target= train_test_split(data,target, test_size=0.2,random_state=42)

#랜덤 포레스트
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42, oob_score=True)

#교차 검증 진행
scores= cross_validate(rt, train_input, train_target, return_train_score=True, n_jobs=-1)
#과적합됨
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

rf.fit(train_input, train_target)
print("결정트리보다 당도 중요도가 감소하고 나머지가 상승 -> 하나의 특성에 과도하게 집중되지 않도록 함")
print(rf.feature_importances_)
# 부트스트랩 샘플 고른거 제외 나머지로 평가도 진행함
print(rf.oob_score_)


0.9973541965122431 0.8905151032797809
결정트리보다 당도 중요도가 감소하고 나머지가 상승 -> 하나의 특성에 과도하게 집중되지 않도록 함
[0.23167441 0.50039841 0.26792718]
0.8934000384837406


##엑스트라 트리
랜덤 포레스트와 차이점은 부트스트랩 샘플을 사용하지 않음. 일부 샘플이 아닌 전체 샘플로 훈련 진행,

하지만 노드 분할시 가장 좋은 분할이 아닌 랜덤 분할함(splitter=random), 성능은 떨어지나 오버피팅을 막고 테스트 점수를 높임,분할에 시간 덜 써서 속도 빠름

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et= ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores =cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


##그레디언트 부스팅
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차 보완

기본적으로 깊이가 3인 트리 100개 사용, 경사 하강법 이용

오버피팅에 강함

랜덤 포레스트보다 좋은 성능 but 훈련 속도 느림(병렬 훈련 불가)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb= GradientBoostingClassifier(random_state=42)
scores= cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 결정 트리 개수 500개로 늘리고 학습률 0.2로늘림(기본 0.1)
gb= GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores= cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999


##히스토그램 기반 그래디언트 부스팅
정형 데이터 다루는 머신러닝 알고리즘 중에 인기 가장 좋음
입력 특성(훈련 데이터)을 256개 구간으로 나눔(히스토그램 구간).

기존 Gradient Boosting에선 정확한 분할 후보를 일일이 탐색하나
이건 연속값을 그대로 사용하지 않고 히스토그램 기반으로 값을 미리 구간으로 나누고 분기 탐색 후 트리 생성

데이터가 크고 빠르게 구하고 싶을 때 사용, 일반 그레디언트보다 오버피팅 억제하면서 좋은 성능을 제공

In [15]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores= cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
# 안에 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)
# n_repeats -> 매개변수 랜덤하게 섞을 확률
result= permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42,n_jobs=-1)
print(result.importances_mean)
print()

result= permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42,n_jobs=-1)
print(result.importances_mean)
hgb.score(test_input, test_target)

############################################################
#히스토그램 기반 그레디언트 부스팅 알고리즘 라이브러리
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores= cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#히스토그램 기반 그레디언트 부스팅 라이브러리(마이크로소프트)
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores= cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]

[0.05969231 0.20238462 0.049     ]
0.9567059184812372 0.8783915747390243
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000627 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000287 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] 